In [1]:
import os.path
import numpy as np
import matplotlib.pyplot as plt
import scipy
import refnx
from refnx.dataset import ReflectDataset, Data1D
from refnx.analysis import Transform, CurveFitter, Objective, Model, Parameter
from refnx.reflect import SLD, Slab, ReflectModel
# from sepObjectiveGO import getObjective as gO
from dynesty import DynamicNestedSampler
from dynesty import plotting as dyplot
import data_in as di
# from CFitter import CFitter
# import make_sim_data as msd
print('refnx: %s\nscipy: %s\nnumpy: %s' % (refnx.version.version,
                   scipy.version.version, np.version.version))


refnx: 0.1.9
scipy: 1.3.0
numpy: 1.16.4


In [2]:
def min_interesting_lnX(res):
    returns = None
    for lnX, logz in zip(res["logvol"], res['logz']):
        if logz >= 0.:
            break
        else:
            returns = -lnX
#     returns = min([-lnX for lnX, wt in zip(res["logvol"], np.exp(res['logwt'] - res['logz'][-1])) if wt>0])
    return returns

# maxLength = 350
def logp_extra(model, data=None):
    returns = 0
    if sum(model.structure.slabs()[1:-1,0])>maxLength:
        returns = -np.inf
    return returns

In [3]:
# data = helper.get_data('./data/bilayer/',
#                            ['54696_97_edit.txt', '54699_00_edit.txt'])
data1 = di.data_in("lipidBilayerData/54696_97_edit.txt")
data2= di.data_in("lipidBilayerData/54699_00_edit.txt")
d2o = SLD(6.07, "D2O")
pmw = SLD(2.35, "Protein Matched Water")
air = SLD(0.00, "Air")

bilayer_d2o = bl.Bilayer(d2o)
bilayer_pmw = bl.Bilayer(pmw)
#bilayer_d2o2 = bl.Bilayer(d2o, name="Bilayer2")
#bilayer_pmw2 = bl.Bilayer(pmw, name="Bilayer2")
dodab_d2o = ml.Dodab(d2o)
dodab_pmw = ml.Dodab(pmw)

structure_d2o = air(0, 0) | dodab_d2o | d2o(
0, 0) | bilayer_d2o | d2o(0, 0) #| bilayer_d2o2 | d2o(0, 0)
structure_pmw = air(0, 0) | dodab_pmw | pmw(
0, 0) | bilayer_pmw | pmw(0, 0) #| bilayer_pmw2 | pmw(0, 0)

d2o.real.setp(6.07, vary=True, bounds=(5.35, 6.35))
pmw.real.setp(2.34, vary=True, bounds=(1.35, 3.35))
# #bilayer_d2o2.presence.setp(0.5, vary=True, bounds=(0.01, 0.99))
# #bilayer_pmw2.presence.setp(constraint=bilayer_d2o2.presence)

# # All of the necessary constraints for the model
# bilayer_d2o.popg.water_per_lipid_head.constraint = (
# bilayer_d2o.popc.water_per_lipid_head)
# bilayer_pmw.popg.water_per_lipid_head.constraint = (
# bilayer_d2o.popc.water_per_lipid_head)
# bilayer_pmw.popc.water_per_lipid_head.constraint = (
# bilayer_d2o.popc.water_per_lipid_head)

structure_d2o[2].thick.setp(14.3, vary=True, bounds=(6, 20))
# # structure_d2o[4].thick.setp(14.3, vary=True, bounds=(4, 40))
structure_d2o[2].rough.setp(4.6, vary=True, bounds=(3, 20))
# #structure_d2o[4].rough.setp(4.6, vary=True, bounds=(3, 20))

# structure_pmw[2].thick.constraint = structure_d2o[2].thick
# structure_pmw[2].rough.constraint = structure_d2o[2].rough

# #structure_pmw[4].thick.constraint = structure_d2o[4].thick
# #structure_pmw[4].rough.constraint = structure_d2o[4].rough

# bilayer_d2o.popc.s_sld.constraint = d2o.real
# bilayer_d2o.popg.s_sld.constraint = d2o.real
# bilayer_pmw.popc.s_sld.constraint = d2o.real
# bilayer_pmw.popg.s_sld.constraint = pmw.real
# dodab_d2o.roughness.constraint = structure_d2o[2].rough
# dodab_pmw.roughness.constraint = structure_d2o[2].rough
# dodab_d2o.apm.constraint = dodab_pmw.apm
# dodab_d2o.water_per_lipid_head.constraint = dodab_pmw.water_per_lipid_head
# bilayer_d2o.roughness_bottom.constraint = bilayer_d2o.roughness_top
# bilayer_pmw.roughness_top.constraint = bilayer_d2o.roughness_top
# bilayer_pmw.roughness_bottom.constraint = bilayer_d2o.roughness_top
# structure_d2o[-1].rough.constraint = bilayer_d2o.roughness_bottom
# structure_pmw[-1].rough.constraint = bilayer_d2o.roughness_bottom
# bilayer_pmw.apm.constraint = bilayer_d2o.apm

# #bilayer_d2o2.popc.s_sld.constraint = d2o.real
# #bilayer_d2o2.popg.s_sld.constraint = d2o.real
# #bilayer_pmw2.popc.s_sld.constraint = d2o.real
# #bilayer_pmw2.popg.s_sld.constraint = pmw.real
# #bilayer_d2o2.roughness_top.constraint = bilayer_d2o.roughness_top
# #bilayer_d2o2.roughness_bottom.constraint = bilayer_d2o.roughness_top
# #bilayer_pmw2.roughness_top.constraint = bilayer_d2o.roughness_top
# #bilayer_pmw2.roughness_bottom.constraint = bilayer_d2o.roughness_top
# #bilayer_pmw2.apm.constraint = bilayer_d2o.apm
# #bilayer_d2o2.apm.constraint = bilayer_d2o.apm
# #bilayer_d2o2.popg.water_per_lipid_head.constraint = (
# #    bilayer_d2o.popc.water_per_lipid_head)
# #bilayer_d2o2.popc.water_per_lipid_head.constraint = (

# #    bilayer_d2o.popc.water_per_lipid_head)
# #bilayer_pmw2.popg.water_per_lipid_head.constraint = (
# #    bilayer_d2o.popc.water_per_lipid_head)
# #bilayer_pmw2.popc.water_per_lipid_head.constraint = (
# #    bilayer_d2o.popc.water_per_lipid_head)


FileNotFoundError: [Errno 2] No such file or directory: 'lipidBilayerData/54696_97_edit.txt'